In [1]:
import TendersWA.Models.HPA as HPA
import tomotopy as tp

In [9]:
corpus = HPA.load_corpus("tenders.hpa.corpus")
if corpus == None:
    docs = []
    import pandas as pd
    
    tenders_structured_path = r"../data/UpdatedAgainTenders.xlsx"
    tenders_structured = pd.read_excel(tenders_structured_path)
    tenders_structured = tenders_structured[["Reference Number", "Contract Title", "Description"]].dropna(subset=["Reference Number"]).drop_duplicates()

    from bs4 import BeautifulSoup
    def remove_html_tags(text):
        soup = BeautifulSoup(text, "html.parser")
        cleaned = soup.get_text().replace('\xa0', ' ')
        cleaned = ' '.join(cleaned.split())
        return cleaned

    for index, row in tenders_structured.iterrows():
        title = row["Contract Title"]
        desc = remove_html_tags(row["Description"])
        docs.append(title + ". " + desc)
    corpus = HPA.create_corpus(docs, "tenders.hpa.corpus")
    
model = tp.LDAModel(tw=tp.TermWeight.ONE, k = 1200, rm_top=0, corpus=corpus)
HPA.train(model, "tenders.hpa.tmm", stopping_factor = 1)

Num docs: 20625 , Vocab size: 24872 , Num words: 614519
Removed top words: []
Training...
Iteration: 00010	ll per word: -10.20941	perplexity: 27157.48797
Iteration: 00020	ll per word: -8.63020	perplexity: 5598.18234
Iteration: 00030	ll per word: -8.21730	perplexity: 3704.48447
Iteration: 00040	ll per word: -8.12805	perplexity: 3388.17985
Iteration: 00050	ll per word: -8.07470	perplexity: 3212.17403
Iteration: 00060	ll per word: -8.01046	perplexity: 3012.28916
Iteration: 00070	ll per word: -7.94205	perplexity: 2813.13149
Iteration: 00080	ll per word: -7.88881	perplexity: 2667.27830
Iteration: 00090	ll per word: -7.84715	perplexity: 2558.44448
Iteration: 00100	ll per word: -7.80719	perplexity: 2458.20697
Iteration: 00110	ll per word: -7.76468	perplexity: 2355.89463
Iteration: 00120	ll per word: -7.73636	perplexity: 2290.13065
Iteration: 00130	ll per word: -7.70819	perplexity: 2226.50810
Iteration: 00140	ll per word: -7.66249	perplexity: 2127.05010
Iteration: 00150	ll per word: -7.63747	p

In [3]:
model = tp.LDAModel.load("tenders.hpa.tmm")

In [ ]:
def get_topic_distribution(model, document_index, n, with_topics=True):
    '''
    model: An HPA model 
    document_index: an index corresponding to the tender list used to train the model
    n: the number of topics you want returned in order of highest to lowest probability
    with_topics: boolean, if you want topic index, or topic name returned
    
    returns the topic distribution for a single document in the form of a list of tupals, of format [(list of topic words,prob),...] 
    '''
    doc = model.docs[document_index]
    topic_probs = doc.get_topics()[:n:]
        
    if with_topics:
        output = []
        for topic_id, prob in topic_probs:
            topic_words = model.get_topic_words(topic_id, top_n=5) 
            words_temp = ", ".join([word for word, weight in topic_words])
            output.append((words_temp,round(prob,4)))
        return output
    else:
        return topic_probs

In [10]:
get_topic_distribution(model, 0, 3)

NameError: name 'get_topic_distribution' is not defined

In [11]:
model.docs[0]

<tomotopy.Document with words="supply delivery aspen bedding material customer requires contractor provide bedding material made aspen must suitable rats mice bedding material product must made entirely aspen species populus tremula">

In [13]:
model.docs[0].get_topics()

[(431, 0.25491848587989807),
 (389, 0.19854453206062317),
 (205, 0.1421596258878708),
 (642, 0.11529677361249924),
 (15, 0.05827947333455086),
 (107, 0.007080479990690947),
 (362, 0.00468446733430028),
 (251, 0.003070002654567361),
 (1121, 0.0028075019363313913),
 (613, 0.0026740864850580692)]

In [14]:
model.summary()

<Basic Info>
| LDAModel (current version: 0.12.5)
| 20625 docs, 614519 words
| Total Vocabs: 24872, Used Vocabs: 24872
| Entropy of words: 7.70177
| Entropy of term-weighted words: 7.70177
| Removed Vocabs: <NA>
|
<Training Info>
| Iterations: 350, Burn-in steps: 0
| Optimization Interval: 10
| Log-likelihood per word: -7.31489
|
<Initial Parameters>
| tw: TermWeight.ONE
| min_cf: 0 (minimum collection frequency of words)
| min_df: 0 (minimum document frequency of words)
| rm_top: 0 (the number of top words to be removed)
| k: 1200 (the number of topics between 1 ~ 32767)
| alpha: [0.1] (hyperparameter of Dirichlet distribution for document-topic, given as a single `float` in case of symmetric prior and as a list with length `k` of `float` in case of asymmetric prior.)
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| seed: 4185686455 (random seed)
| trained in version 0.12.5
|
<Parameters>
| alpha (Dirichlet prior on the per-document topic distributions)
|  [7.61

In [20]:
from sklearn.decomposition import LatentDirichletAllocation

docs = []
import pandas as pd
    
tenders_structured_path = r"../data/UpdatedAgainTenders.xlsx"
tenders_structured = pd.read_excel(tenders_structured_path)
tenders_structured = tenders_structured[["Reference Number", "Contract Title", "Description"]].dropna(subset=["Reference Number"]).drop_duplicates()

from bs4 import BeautifulSoup
def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    cleaned = soup.get_text().replace('\xa0', ' ')
    cleaned = ' '.join(cleaned.split())
    return cleaned

for index, row in tenders_structured.iterrows():
    title = row["Contract Title"]
    desc = remove_html_tags(row["Description"])
    docs.append(title + ". " + desc)

In [113]:
import sklearn.feature_extraction.text.TfidfVectorizer
tender_tfidf = TfidfVectorizer(stop_words="english",
                           min_df=3)
tfidf = tender_tfidf.fit_transform(docs)
np.shape(tfidf[1])

(1, 9574)

In [124]:
lda = LatentDirichletAllocation(
    n_components=100,
    max_iter=20,
    learning_method="online",
)

In [125]:
lda.fit(tfs)

LatentDirichletAllocation(learning_method='online', max_iter=20,
                          n_components=100)

In [130]:
import numpy as np
topic_dist = lda.transform(tfs[0])

In [131]:
np.argmax(topic_dist)

32

In [133]:
topic = lda.components_[np.argmax(topic_dist)]
top_features_ind = topic.argsort()[-10:]
top_features = tf_feature_names[top_features_ind]
print(top_features)

['facia' 'directories' '7500kg' '42835' '6560' 'della' '007' 'eliminating'
 'clearing' 'accp']


In [134]:
docs[0]

'Supply and Delivery of Aspen Bedding Material. The Customer requires a Contractor to provide bedding material made from Aspen and must be suitable for its rats and mice. The bedding material product must be made entirely from the Aspen species Populus tremula.'